### PREPARATION

In [1]:
from pulp import *
import numpy as np
import pandas as pd
import re
import os
import sys
from functools import partial
import time
from itertools import repeat

In [2]:
## Change directory
os.chdir("/home/ziya/Desktop/PROCESS/Optimization/scenarios/scenario-1")

## Read data
var_data = pd.read_csv('./variable-coefficients/input/district/DoktarVariableInput_1_225.csv', encoding = 'utf-8')
min_area_cons = pd.read_csv('./constraints/DoktarDistrictMinAreaCons_scenario-1.csv', encoding = 'utf-8')
max_area_cons = pd.read_csv('./constraints/DoktarDistrictMaxAreaCons_scenario-1.csv', encoding = 'utf-8')
program_name = 'TrAgriOpt'

## Create the LP object, set up as a maximization problem
prob = pulp.LpProblem(program_name, LpMaximize)

In [ ]:
# Report Execution Time
with open("./results/report/Optimization_Report_" + "TEST" + ".txt", "w") as f:
    f.write(str(var_data.info(verbose=False, memory_usage="deep")))

### VARIABLE REGISTRATION
x0 + x1 + x2 + x3 ...

In [3]:
def register_var(rownum = 0):
    variable = str('x' + str(rownum))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat= 'Integer') #make variables binary
    return variable

In [4]:
%%time
# Create decision - sown or not?
row_list = list(var_data.index)
decision_variables = list(map(register_var, row_list))

CPU times: user 7.28 ms, sys: 0 ns, total: 7.28 ms
Wall time: 7.2 ms


In [5]:
print("Total number of decision_variables: " + str(len(decision_variables)))
print(decision_variables[3])
print(type(decision_variables[3]))

Total number of decision_variables: 2023
x3
<class 'pulp.pulp.LpVariable'>


In [6]:
## Add variable names to main csv
var_data['VarName'] = decision_variables

In [7]:
var_data.head()

,CityId,DistrictId,FieldId,IrrigationId,CityName,DistrictName,FieldArea,Percent,Yield,Price,Cost,CropName,Coef,VarName
0,1,225,81751,1,ADANA,ALADAĞ,1.183187,1.079264,292,2.06,798,Wheat,-1760.594052,x0
1,1,225,81752,1,ADANA,ALADAĞ,1.099758,1.079264,292,2.06,798,Wheat,-1636.451155,x1
2,1,225,81753,1,ADANA,ALADAĞ,0.442404,0.979264,292,2.06,798,Wheat,-924.416969,x2
3,1,225,81754,1,ADANA,ALADAĞ,0.791450,1.079274,292,2.06,798,Wheat,-1177.637603,x3
4,1,225,81755,1,ADANA,ALADAĞ,1.012276,0.979274,292,2.06,798,Wheat,-2115.119599,x4


### OPTIMIZATION FORMULA
10*x0 + 5*x1 + 7*x2 - 3*x4 ...

In [8]:
%%time
prob += lpSum(var_data['Coef'] * var_data['VarName'])

CPU times: user 22.9 ms, sys: 3.81 ms, total: 26.7 ms
Wall time: 28.3 ms


### CONSTRAINT ON EACH FIELD ID

x0 + x1 + x2 ... <= 1

x17 + x18 + x19 ... <= 1

...

In [9]:
# Unique FieldId List
id_unqiue = var_data.FieldId.unique()

In [10]:
def add_field_constraint(field_id, prob):
    prob += lpSum(var_data[var_data['FieldId']==field_id]['VarName'].tolist()) <= 1

In [11]:
# starting time
start = time.time()
add_field_partial = partial(add_field_constraint, prob = prob)
list(map(add_field_partial, id_unqiue))
# ending time
end = time.time()
print(f"Runtime of the field constraint registration is {(end - start) / 60} mins")
# Report Optimization State
with open("./results/report/Optimization_Report_TEST.txt", "w") as f:
    print(prob, file=f)

Runtime of the field constraint registration is 0.0012625892957051595 mins


%%time
for i in id_unqiue:
    prob += lpSum(var_data[var_data['FieldId']==i]['VarName'].tolist()) <= 1

### CONSTRAINT ON MIN - MAX AREAS

In [ ]:
def add_area_constraint(district_id, crop_name, prob, var_data, min_area_cons, max_area_cons):
        sub_min_data = min_area_cons[(min_area_cons['CityId'] == 1) & (min_area_cons['DistrictId'] == district_id)]
        sub_max_data = max_area_cons[(max_area_cons['CityId'] == 1) & (max_area_cons['DistrictId'] == district_id)]
        sub_crop_data = var_data[var_data['CropName'] == crop_name]
        min_area = sub_min_data[str('Min' + crop_name + 'Area')]
        max_area = sub_max_data[str('Max' + crop_name + 'Area')]
        prob += pulp.lpSum(sub_crop_data['VarName'] * sub_crop_data['FieldArea']) >= min_area
        prob += pulp.lpSum(sub_crop_data['VarName'] * sub_crop_data['FieldArea']) <= max_area

In [ ]:
# starting time
start = time.time()
# Get District & Crop Name List
district_list = list(var_data.DistrictId.unique())
district_count = len(district_list)
district_list = [x for item in district_list for x in repeat(item, 17)]
crop_list = list(var_data.CropName.unique()) * district_count
add_area_partial = partial(add_area_constraint, prob = prob, var_data = var_data, min_area_cons = min_area_cons, max_area_cons = max_area_cons)
list(map(add_area_partial, district_list, crop_list))
# ending time
end = time.time()
print(f"Runtime of the field constraint registration is {(end - start) / 60} mins")

In [12]:
# Get District & Crop Name List
district_list = list(var_data.DistrictId.unique())
crop_list = list(var_data.CropName.unique())

# Set constraints for each district at Min & Max
for district in district_list:
    sub_min_data = min_area_cons[(min_area_cons['CityId'] == 1) & (min_area_cons['DistrictId'] == district)]
    sub_max_data = max_area_cons[(max_area_cons['CityId'] == 1) & (max_area_cons['DistrictId'] == district)]
    for crop in crop_list:
        sub_crop_data = var_data[var_data['CropName'] == crop]
        min_area = sub_min_data[str('Min' + crop + 'Area')]
        max_area = sub_max_data[str('Max' + crop + 'Area')]
        prob += pulp.lpSum(sub_crop_data['VarName'] * sub_crop_data['FieldArea']) >= min_area
        prob += pulp.lpSum(sub_crop_data['VarName'] * sub_crop_data['FieldArea']) <= max_area

### RUN PROGRAM

In [22]:
listSolvers(onlyAvailable=True)

['PULP_CBC_CMD', 'PULP_CHOCO_CMD']

In [23]:
solver = getSolver('PULP_CBC_CMD', timeLimit=200)

In [24]:
%%time
# now run optimization
optimization_result = prob.solve(solver)
print("Status:", LpStatus[prob.status])
print("Optimal Solution to the problem: ", value(prob.objective))

Status: Optimal
Optimal Solution to the problem:  3071560.608431639
CPU times: user 28.7 ms, sys: 8.01 ms, total: 36.7 ms
Wall time: 81.4 ms


### WRITE RESULTS

In [ ]:
var_data['VarValue'] = var_data['VarName'].apply(lambda x: x.varValue)
var_data.head(17)

In [ ]:
var_data.to_csv("./results/scenario-1/doktar-opt-v01.csv", encoding = "utf-8-sig", index = False)

### TESTING CODE

In [ ]:
import glob
import re
input_c_path = "/home/ziya/Desktop/PROCESS/Optimization/scenarios/scenario-1/variable-coefficients/input/city/"
input_d_path = "/home/ziya/Desktop/PROCESS/Optimization/scenarios/scenario-1/variable-coefficients/input/district/"
# Listing files
input_c_list = glob.glob(input_c_path + '*.csv')
input_c_name_list = list(map(lambda name: name.split("/")[-1], input_c_list))
id_list = list(map(lambda file_name: re.findall(r'(\d+)', file_name), input_c_name_list))
flat_id_list = [item for sublist in id_list for item in sublist]

# Get iteration lists
city_id_list = list(map(int, flat_id_list))

print(city_id_list)

# Listing files
input_d_list = glob.glob(input_d_path + 'DoktarVariableInput_' + str(city_id_list[0]) + '*.csv')
input_d_name_list = list(map(lambda name: name.split("_")[-1].split(r".")[0], input_d_list))
input_d_name_list
id_list = list(map(lambda name: name.rsplit("_", -1), input_d_name_list))
flat_id_list = [item for sublist in id_list for item in sublist]

# Get iteration lists
district_id_list = list(map(int, flat_id_list))
district_id_list

In [ ]:
# Slicing Lists
import numpy as np
input_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
input_list = np.array(input_list)
input_list = np.array_split(input_list, 5)
input_list = [l.tolist() for l in input_list]
input_list

### PREPROCESSING

In [ ]:
import pandas as pd
import os
from functools import partial

def add_columns(base_name, yield_path, price_path, cost_path, scenario_name):
    
    # Read Base Data
    base_data = pd.read_csv(base_name, usecols=["CityId", "DistrictId", "FieldId",
    "IrrigationId", "FieldArea", "WheatPercent", "BarleyPercent", "SunflowerPercent",
    "RicePercent", "LentilPercent", "ChickpeaPercent", "CottonPercent", "SugarbeetPercent",
    "SoybeanPercent", "PeanutPercent", "OatPercent", "PotatoPercent", "CornPercent",
    "CanolaPercent", "OnionPercent", "BeanPercent", "TomatoPercent"],
    dtype={"CityId": "uint8", "DistrictId": "uint16", "FieldId": "uint64",
    "IrrigationId": "uint8", "FieldArea": "float32"})
    
    # Add Yield Columns
    yield_data = pd.read_csv(yield_path + "DoktarVariableYield_" + scenario_name + ".csv", encoding="latin-1",
    dtype={"CityId": "uint8", "DistrictId": "uint16"})
    base_data = pd.merge(base_data, yield_data, on = 'DistrictId', how = 'left', suffixes = ('', '_extra'))
    base_data = base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]
    
    # Add Price Columns
    price_data = pd.read_csv(price_path + "DoktarVariablePrice_" + scenario_name + ".csv", encoding = "latin-1",
    dtype={"CityId": "uint8", "DistrictId": "uint16"})
    base_data = pd.merge(base_data, price_data, on = 'DistrictId', how = 'left', suffixes = ('', '_extra'))
    base_data = base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]

    # Add Cost Columns
    cost_data = pd.read_csv(cost_path + "DoktarVariableCost_" + scenario_name + ".csv", encoding = "latin-1",
    dtype={"CityId": "uint8", "DistrictId": "uint16", "IrrigationId": "uint8"})
    base_data = pd.merge(base_data, cost_data, on = ['DistrictId', 'IrrigationId'], how = 'left', suffixes = ('', '_extra'))
    base_data= base_data[base_data.columns.drop(list(base_data.filter(regex='_extra')))]
    
    return base_data


def wide_to_long(crop_name, base_data):
    main_columns = ['CityId', 'DistrictId', 'FieldId', "IrrigationId", 'FieldArea']
    crop_columns = list(base_data.filter(regex=(crop_name)).columns)
    base_data = base_data.filter(main_columns + crop_columns)
    base_data = base_data.rename(columns={crop_name + 'Percent': 'Percent', crop_name + 'Yield': 'Yield', crop_name + 'Price': 'Price', crop_name + 'Cost': 'Cost'})
    base_data['CropName'] = crop_name

    return base_data


def wtl_map(base_data):
    crop_list = ['Wheat', 'Barley', 'Oat',
    'Cotton', 'Soybean', 'Tomato',
    'Canola', 'Potato', 'Sugarbeet',
    'Onion', 'Bean', 'Lentil', 'Chickpea',
    'Peanut', 'Rice', 'Corn', 'Sunflower']

    # Partial wide_to_long Function
    wide_func = partial(wide_to_long, base_data = base_data)
    # Map to each Crop
    long_list = list(map(wide_func, crop_list))
    long_data = pd.concat(long_list)

    return long_data


def add_coef(base_data, target = "Profit"):
    if target == "Profit":
        base_data['Coef'] = base_data['FieldArea'] * 10 * base_data['Percent'] * base_data['Yield'] * base_data['Price'] - base_data['FieldArea'] * 10 * base_data['Cost']
        return(base_data)
    elif target == "Physical":
        base_data['Coef'] = base_data.eval('Percent')
        return(base_data)
    else:
        print("Hayat kısa, kuşlar uçuyor...")


def write_input_d(city_id, city_data, input_path):
    for district_id in city_data.DistrictId.unique():
        district_data = city_data[city_data['DistrictId'] == district_id]
        district_data.to_csv(input_path + "DoktarVariableInput_" + str(city_id) + "_" + str(district_id) + ".csv",
        encoding = "utf-8-sig",
        index = False)


def write_input_c(city_id, city_data, input_path):
    city_data.to_csv(input_path + "DoktarVariableInput_" + str(city_id) + ".csv",
    encoding = "utf-8-sig",
    index = False)


def do_preprocess(base_name, city_id, yield_path, price_path, cost_path, scenario_name, target, input_c_path, input_d_path):
    
    # Create base_data add columns
    base_data = add_columns(base_name = base_name, yield_path = yield_path, price_path = price_path, cost_path = cost_path, scenario_name = scenario_name)
    # Wide to Long
    base_data = wtl_map(base_data = base_data)
    # Add Coefs
    base_data = add_coef(base_data = base_data, target = target)
    # Write Cities
    write_input_c(city_id = city_id, city_data = base_data, input_path = input_c_path)
    write_input_d(city_id = city_id, city_data = base_data, input_path = input_d_path)

    del(base_data)

In [ ]:
# File paths
main_path = "/home/ziya/Desktop/PROCESS/Optimization/scenarios"
base_path = './variable-coefficients/base/'
input_c_path = './variable-coefficients/input/city/'
input_d_path = './variable-coefficients/input/district/'
price_path = './variable-coefficients/price/'
yield_path = './variable-coefficients/yield/'
cost_path = './variable-coefficients/cost/'

In [ ]:
scenario_list = ['scenario-2']
coef_list = ['Physical']

scenario_name = 'scenario-2'

# Scenario Dict
sc_dict = {scenario_list[i]: coef_list[i] for i in range(len(scenario_list))}
# Scenario Specification
scenario_path = main_path + "/" + scenario_name
# Set scenario path
os.chdir(scenario_path)

# base list prepreation
base_list = glob.glob(base_path + '*.csv')
base_name_list = list(map(lambda name: name.split("/")[-1], base_list))
id_list = list(map(lambda file_name: re.findall(r'(\d+)', file_name), base_name_list))
flat_id_list = [item for sublist in id_list for item in sublist]

# Get iteration lists
city_id_list = list(map(int, flat_id_list))

# Set first iter
base_name = base_list[0]
city_id = city_id_list[0]

# Create base_data add columns
base_data = add_columns(base_name = base_name, yield_path = yield_path, price_path = price_path, cost_path = cost_path, scenario_name = scenario_name)

base_data = wtl_map(base_data)
base_data = add_coef(base_data, target = sc_dict.get(scenario_name))
for district_id in base_data.DistrictId.unique():
    print(district_id)

In [ ]:
type(float(10))